%pip install geopy > NUL
%pip install osmnx > NUL

In [25]:
import math
import copy
import json
import osmnx as ox
import geopy.distance

In [26]:
# read raw data...
with open('raw_data.json', 'r') as f:
    data = json.load(f)

# 2 way converting...
n = len(data['listLinks']);
for i in range(n-1, -1, -1):
    listLinks = data['listLinks'][i]
    for j in listLinks:
        # append front
        data['listLinks'][j].append(i)

for i in range (len(data['listLinks'])):
    data['listLinks'][i].sort()

# 1 way tuning...
for source, dist in data['list1Ways']:
    data['listLinks'][dist].remove(source)

# Slice

In [27]:
# slice
data['listExpandListNodes'] = copy.deepcopy(data['listNodes'])
data['listExpandListLinks'] = copy.deepcopy(data['listLinks'])
n = len(data['listLinks']);
data['listExpandSourceNode'] =  [[] for _ in range(n)]

for i in range(n):
    listLinks = data['listLinks'][i]
    for j in listLinks:
        if i in data['listLinks'][j]:
            if (i > j): continue

        source = (data['listNodes'][i]['lat'], data['listNodes'][i]['lng'])
        destination = (data['listNodes'][j]['lat'], data['listNodes'][j]['lng'])

        n_slice = math.floor(
            geopy.distance.geodesic(source, destination).m / 1
        )
        
        for i_slice in range(1, n_slice):
            lat = (i_slice/n_slice) * destination[0] + (1 - i_slice/n_slice) * source[0]
            lng = (i_slice/n_slice) * destination[1] + (1 - i_slice/n_slice) * source[1]

            point = {"lat": lat, "lng": lng}
            data['listExpandListNodes'].append(point)
        
            if i in data['listLinks'][j]:
                # node that this node can go to
                data['listExpandListLinks'].append([i, j])
                # node can go to this node
                data['listExpandSourceNode'].append([i, j])
            else: 
                # node that this node can go to
                data['listExpandListLinks'].append([j])
                # node can go to this node
                data['listExpandSourceNode'].append([i])

print('listExpandListNodes:', len(data['listExpandListNodes']))
print('listExpandSourceNode:', len(data['listExpandSourceNode']))

listExpandListNodes: 8575
listExpandSourceNode: 8575


In [28]:
# ...and write
with open("../map/data.js", "w") as f:
    f.write('const DATA = ')

with open("../map/data.js", "a") as f:
    json.dump(data, f)